In [13]:
# librerias
import pandas as pd
import numpy as np

In [2]:
import joblib

def load_df(path):
    df = joblib.load(path)
    return df

In [8]:
df = load_df('../tmp/fe_df.pkl')

In [9]:
df

,fisica,virtual,no_prime,si_prime,f,m,no_def,NA_establecimiento,abarrotes,farmacia,...,dcto,cashback,linea_tc,interes_tc,sin_dia,cos_dia,sin_hora,cos_hora,fraude,fecha
0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.263136,-0.176315,0.389427,0.289465,-9.510565e-01,-0.309017,-0.866025,0.500000,0,2020-01-21
1,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.508714,-1.006380,0.389427,0.289465,5.665539e-16,-1.000000,0.965926,-0.258819,0,2020-01-15
2,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.508714,2.139704,1.440304,0.811001,-4.067366e-01,-0.913545,0.258819,-0.965926,0,2020-01-17
3,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,-0.508714,0.316006,1.440304,0.811001,8.660254e-01,0.500000,0.258819,-0.965926,0,2020-01-05
4,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,-0.508714,-0.834270,1.440304,0.811001,-1.133108e-15,1.000000,0.500000,-0.866025,0,2020-01-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26970,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.508714,0.269322,-1.026972,0.811001,9.945219e-01,-0.104528,-0.258819,-0.965926,0,2020-01-08
26971,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.508714,0.708078,-1.026972,0.811001,2.079117e-01,-0.978148,-0.707107,-0.707107,0,2020-01-14
26972,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.508714,-0.277543,-1.026972,0.811001,-2.079117e-01,-0.978148,0.258819,-0.965926,0,2020-01-16
26973,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,...,-0.438280,-1.355553,-1.026972,0.811001,4.067366e-01,-0.913545,-0.965926,-0.258819,0,2020-01-13


# Train and test data

In [11]:
df = df.sort_values(by= ['fecha'])

In [14]:
n = 0.80*df.shape[0]
n = np.round(n).astype(int)

In [15]:
df_train = df.iloc[0:n,:]
df_test = df.iloc[n:,:]

In [17]:
X_train = df_train.drop(['fraude','fecha'],1)
y_train = df_train['fraude'].values
X_test = df_test.drop(['fraude','fecha'],1)
y_test = df_test['fraude'].values

In [18]:
print('X_train shape: ', X_train.shape)
print('y_train shape: ', y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', y_test.shape)

X_train shape:  (21580, 27)
y_train shape:  (21580,)
X_test shape:  (5395, 27)
y_test shape:  (5395,)


In [21]:
y_train.sum()

654

In [22]:
y_test.sum()

156

# Magic loop

## Definimos los modelos:

In [26]:
algorithms = ['tree', 'logistic']

In [27]:
algorithms_dict = {'tree': 'tree_grid_search',
                  'logistic': 'logitstic_regression'}

In [28]:
grid_search_dict = {'tree_grid_search': {'max_depth': [3, 5], 
                                         'min_samples_leaf': [3, 5]},
                   'logitstic_regression': {'penalty':['l2'],
                                           'solver':['sag','saga']}}

In [31]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

estimators_dict = {'tree': DecisionTreeClassifier(random_state=96356),
                   'logistic': LogisticRegression(random_state=928)}

In [32]:
best_estimators = []

In [33]:
features = X_train
labels = y_train

In [34]:
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV

for algorithm in algorithms:
        estimator = estimators_dict[algorithm]
        grid_search_to_look = algorithms_dict[algorithm]
        grid_params = grid_search_dict[grid_search_to_look]
        
        tscv = TimeSeriesSplit(
            n_splits = 3
        )
        gs = GridSearchCV(
            estimator,
            grid_params, 
            scoring = 'precision', 
            cv = tscv, 
            n_jobs = -1
        )
        
        #train
        gs.fit(features, labels)
        #best estimator
        best_estimators.append(gs)

/Users/ele/.pyenv/versions/dpa_env/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:329: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [35]:
best_estimators

[GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
              estimator=DecisionTreeClassifier(random_state=96356), n_jobs=-1,
              param_grid={'max_depth': [3, 5], 'min_samples_leaf': [3, 5]},
              scoring='precision'),
 GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
              estimator=LogisticRegression(random_state=928), n_jobs=-1,
              param_grid={'penalty': ['l2'], 'solver': ['sag', 'saga']},
              scoring='precision')]

In [39]:
models = best_estimators

In [40]:
models[0]

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
             estimator=DecisionTreeClassifier(random_state=96356), n_jobs=-1,
             param_grid={'max_depth': [3, 5], 'min_samples_leaf': [3, 5]},
             scoring='precision')

In [41]:
models[1]

GridSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
             estimator=LogisticRegression(random_state=928), n_jobs=-1,
             param_grid={'penalty': ['l2'], 'solver': ['sag', 'saga']},
             scoring='precision')

In [43]:
predicted_scores_model_1 = models[1].predict_proba(X_test) # logistic
predicted_scores_model_0 = models[0].predict_proba(X_test) # árbol

In [44]:
punto_corte = 0.7

In [45]:
predicted_scores_model_1

array([[0.96880206, 0.03119794],
       [0.97243717, 0.02756283],
       [0.97750212, 0.02249788],
       ...,
       [0.97186991, 0.02813009],
       [0.96841289, 0.03158711],
       [0.97256462, 0.02743538]])

In [46]:
predicted_scores_model_0

array([[0.97610224, 0.02389776],
       [0.97610224, 0.02389776],
       [0.97610224, 0.02389776],
       ...,
       [0.97610224, 0.02389776],
       [0.96582198, 0.03417802],
       [0.96582198, 0.03417802]])

In [48]:
from sklearn.metrics import precision_score

In [49]:
labels_model_1 = [0 if score < punto_corte else 1 for score in predicted_scores_model_1[:,1]]

In [50]:
labels_model_0 = [0 if score < punto_corte else 1 for score in predicted_scores_model_0[:,1]]

In [54]:
precision_0 = precision_score(y_test, labels_model_0,average='micro')

In [53]:
precision_1 = precision_score(y_test, labels_model_1, average='micro')

In [55]:
mod_0 = pd.DataFrame(labels_model_0)
mod_0.columns = ['fraude']

In [56]:
mod_0

,fraude
0,0
1,0
2,0
3,0
4,0
...,...
5390,0
5391,0
5392,0
5393,0


In [57]:
mod_1 = pd.DataFrame(labels_model_1)
mod_1.columns = ['results']

In [58]:
model_0 = pd.concat([X_test.reset_index(drop='True'), mod_0.reset_index(drop='True')], axis=1)
model_1 = pd.concat([X_test.reset_index(drop='True'), mod_1.reset_index(drop='True')], axis=1)

In [59]:
precision_0

0.9710843373493976

In [60]:
precision_1

0.9710843373493976